In [1]:
! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [2]:
! pip install torch-summary==1.4.5
! pip install matplotlib==3.9.3

In [ ]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image

ROOT = './data'
train_data = datasets.MNIST(
    root=ROOT,
    train=True,
    download=True
)
test_data = datasets.MNIST(
    root=ROOT,
    train=False,
    download=True
)


In [4]:
valid_ratio = 0.9

len_train = int(len(train_data) * valid_ratio)
len_valid = len(train_data) - len_train

In [5]:
train_data, valid_data = data.random_split(train_data, [len_train, len_valid])

In [ ]:
mean  = train_data.dataset.data.float().mean() / 255
std = train_data.dataset.data.float().std() / 255

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean, ), (std), )
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean, ), (std), )
])

train_data.dataset.transform = train_transforms
valid_data.dataset.transform = valid_transforms

batch_size = 256
train_dataloader = data.DataLoader(
    train_data, 
    shuffle=True,
    batch_size=batch_size, 
    num_workers=2
)

valid_dataloader = data.DataLoader(
    valid_data,
    shuffle=True,
    batch_size=batch_size,
    num_workers=2
)

In [ ]:
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes) -> None:
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels=1, out_channels=6, kernel_size=5, padding='same'
        )
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(16*4*4, 120)
        self.linear2 = nn.Linear(120, 84)
        self.out = nn.Linear(84, num_classes)
    
    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.linear1(outputs)
        outputs = self.linear2(outputs)
        outputs = self.out(outputs)
        return outputs